In [1]:
!pip install torchvision

  Using cached torchvision-0.25.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (5.4 kB)
  Using cached numpy-2.4.2-cp312-cp312-macosx_14_0_arm64.whl.metadata (6.6 kB)
  Using cached torch-2.10.0-2-cp312-none-macosx_11_0_arm64.whl.metadata (31 kB)
  Using cached pillow-12.1.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (8.8 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached setuptools-82.0.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.6.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2026.2.0-py3-none-any.whl.metadata (10 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached markupsafe-3.0.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.7 kB)
Using cached torchvision-0.25.0-cp312-cp312-macosx_11_0_arm64.whl (1.9 MB)
Using cached torch-2.10.0-2-cp312-none-macosx_11_0_arm

In [4]:
!pip install torchmetrics

  Using cached torchmetrics-1.8.2-py3-none-any.whl.metadata (22 kB)
  Using cached lightning_utilities-0.15.2-py3-none-any.whl.metadata (5.7 kB)
Using cached torchmetrics-1.8.2-py3-none-any.whl (983 kB)
Using cached lightning_utilities-0.15.2-py3-none-any.whl (29 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [torchmetrics] [torchmetrics]


In [1]:
!pip install numpy

In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%cd "/Users/Marmarkabiri/Desktop/Downloads/ML_Projectt"

[Errno 2] No such file or directory: '/Users/Marmarkabiri/Desktop/Downloads/ML_Projectt'
/Users/marmarkabiri/Downloads/ML_Projectt


In [32]:
import os
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

import config
from dataset.face_dataset import FaceDataset
from models.unet import UNet
from losses.losees import get_loss, get_identity_loss
from utils.device import get_device

In [33]:
def save_curves(train_losses: list[float], val_losses: list[float]) -> None:
    os.makedirs(config.RESULTS_DIR, exist_ok=True)
    plt.figure()
    plt.plot(train_losses, label="Train L1 Loss")
    plt.plot(val_losses, label="Val L1 Loss")
    plt.legend()
    plt.xlabel("Epoch")
    plt.savefig(os.path.join(config.RESULTS_DIR, "loss_curve.png"))
    plt.close()

In [40]:
@torch.no_grad()
def compute_l1_loss(model, loader, criterion, device):
    model.eval()
    total = 0.0
    for degraded, clean in loader:
        degraded = degraded.to(device)
        clean = clean.to(device)
        output = model(degraded)
        total += criterion(output, clean).item()
    return total / len(loader)

In [41]:
device = get_device()

use_align = getattr(config, "USE_ALIGNMENT", False)
img_size = getattr(config, "IMG_SIZE", 128)
train_dataset = FaceDataset(config.DATA_TRAIN_DIR, config.DESTRUCTION, use_alignment=use_align, img_size=img_size)
train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)

val_dataset = FaceDataset(config.DATA_VAL_DIR, config.DESTRUCTION, use_alignment=use_align, img_size=img_size)
val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, shuffle=False)

model = UNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=config.LR)
criterion = get_loss()
identity_loss_fn = get_identity_loss(device) if getattr(config, "USE_IDENTITY_LOSS", False) else None

best_loss = float("inf")
train_losses = []
val_losses = []

patience = getattr(config, "EARLY_STOPPING_PATIENCE", 5)
min_delta = getattr(config, "EARLY_STOPPING_MIN_DELTA", 0.05)
epochs_no_improve = 0

for epoch in range(config.EPOCHS):
    model.train()
    epoch_loss = 0.0

    for degraded, clean in train_loader:
        degraded = degraded.to(device)
        clean = clean.to(device)

        output = model(degraded)
        loss = criterion(output, clean)
        if identity_loss_fn is not None:
            loss = loss + getattr(config, 'IDENTITY_LOSS_WEIGHT', 0.1) * identity_loss_fn(output, clean)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(train_loader)
    train_losses.append(epoch_loss)

    val_loss_epoch = compute_l1_loss(model, val_loader, criterion, device)
    val_losses.append(val_loss_epoch)

    print(
        f"Epoch {epoch + 1}/{config.EPOCHS} | "
        f"Train L1 Loss: {epoch_loss:.4f} | "
        f"Val L1 Loss: {val_loss_epoch:.4f}"
    )

    # Save best model + early stopping (L1: lower is better)
    if val_loss_epoch < best_loss :
        best_loss = val_loss_epoch
        torch.save(model.state_dict(), config.MODEL_PATH)
        print("Best model saved.")
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        print(f"No improvement for {epochs_no_improve}/{patience} epochs.")
        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            break

    save_curves(train_losses, val_losses)

# final save curves (in case loop ended early)
save_curves(train_losses, val_losses)

Epoch 1/20 | Train L1 Loss: 0.2640 | Val L1 Loss: 0.2549
Best model saved.
Epoch 2/20 | Train L1 Loss: 0.2211 | Val L1 Loss: 0.1592
Best model saved.
Epoch 3/20 | Train L1 Loss: 0.1135 | Val L1 Loss: 0.1009
Best model saved.
Epoch 4/20 | Train L1 Loss: 0.0936 | Val L1 Loss: 0.0888
Best model saved.
Epoch 5/20 | Train L1 Loss: 0.0852 | Val L1 Loss: 0.0851
Best model saved.
Epoch 6/20 | Train L1 Loss: 0.0812 | Val L1 Loss: 0.0836
Best model saved.
Epoch 7/20 | Train L1 Loss: 0.0788 | Val L1 Loss: 0.0756
Best model saved.
Epoch 8/20 | Train L1 Loss: 0.0765 | Val L1 Loss: 0.0741
Best model saved.
Epoch 9/20 | Train L1 Loss: 0.0756 | Val L1 Loss: 0.0758
No improvement for 1/5 epochs.
Epoch 10/20 | Train L1 Loss: 0.0731 | Val L1 Loss: 0.0693
Best model saved.
Epoch 11/20 | Train L1 Loss: 0.0702 | Val L1 Loss: 0.0691
Best model saved.
Epoch 12/20 | Train L1 Loss: 0.0686 | Val L1 Loss: 0.0662
Best model saved.
Epoch 13/20 | Train L1 Loss: 0.0658 | Val L1 Loss: 0.0649
Best model saved.
Epoch 14